# Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import cufflinks as cf
cf.go_offline()
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
balls = pd.read_csv('../input/ipl-2008-to-2021-all-match-dataset/IPL_Ball_by_Ball_2008_2022.csv')
balls.shape

In [ ]:
matches = pd.read_csv('../input/ipl-2008-to-2021-all-match-dataset/IPL_Matches_2008_2022.csv')
matches.shape

In [ ]:
balls.head()

In [ ]:
balls.info()

In [ ]:
balls.describe()

In [ ]:
matches.head()

In [ ]:
matches.info()

# Data Visualization

In [ ]:
matches['City'].value_counts().iplot()

# EDA and Feature Engineering

## Finding total score of the innings

In [ ]:
total_score = balls.groupby(['ID', 'innings']).sum()['total_run'].reset_index()

In [ ]:
total_score.head()

## We only need score of 1st innings
our target is winner prediction so we only need the score of first innings

In [ ]:
total_score = total_score[total_score['innings']==1]

In [ ]:
total_score.head()

In [ ]:
fig = px.histogram(total_score,nbins=30, x='total_run')
fig.show()

In [ ]:
total_score['target'] = total_score['total_run'] + 1

Merge with the mathces dataset

In [ ]:
match_df = matches.merge(total_score[['ID','target']], on='ID')

In [ ]:
match_df.head()

## Removing old teams / updating teams new names

In [ ]:
match_df['Team1'].unique()

In [ ]:
teams = [
    'Rajasthan Royals',
    'Royal Challengers Bangalore',
    'Sunrisers Hyderabad', 
    'Delhi Capitals', 
    'Chennai Super Kings',
    'Gujarat Titans', 
    'Lucknow Super Giants', 
    'Kolkata Knight Riders',
    'Punjab Kings', 
    'Mumbai Indians'
]

In [ ]:
match_df['Team1'] = match_df['Team1'].str.replace('Delhi Daredevils', 'Delhi Capitals')
match_df['Team2'] = match_df['Team2'].str.replace('Delhi Daredevils', 'Delhi Capitals')
match_df['WinningTeam'] = match_df['WinningTeam'].str.replace('Delhi Daredevils', 'Delhi Capitals')

match_df['Team1'] = match_df['Team1'].str.replace('Kings XI Punjab', 'Punjab Kings')
match_df['Team2'] = match_df['Team2'].str.replace('Kings XI Punjab', 'Punjab Kings')
match_df['WinningTeam'] = match_df['WinningTeam'].str.replace('Kings XI Punjab', 'Punjab Kings')


match_df['Team1'] = match_df['Team1'].str.replace('Deccan Chargers', 'Sunrisers Hyderabad')
match_df['Team2'] = match_df['Team2'].str.replace('Deccan Chargers', 'Sunrisers Hyderabad')
match_df['WinningTeam'] = match_df['WinningTeam'].str.replace('Deccan Chargers', 'Sunrisers Hyderabad')

In [ ]:
match_df = match_df[match_df['Team1'].isin(teams)]
match_df = match_df[match_df['Team2'].isin(teams)]
match_df = match_df[match_df['WinningTeam'].isin(teams)]

In [ ]:
match_df.shape

In [ ]:
match_df.columns

In [ ]:
fig = px.histogram(match_df, x='WinningTeam')
fig.show()

## We want only the matches where D/L is not applied 
Removing all matches effected due to rain

In [ ]:
match_df['method'].unique()

In [ ]:
match_df['method'].value_counts()

In [ ]:
match_df = match_df[match_df['method'].isna()]

In [ ]:
match_df.shape

In [ ]:
match_df.columns

In [ ]:
match_df = match_df[['ID','City','Team1','Team2','WinningTeam','target']].dropna()

In [ ]:
match_df.head()

In [ ]:
match_df.isna().sum()

Merge the match_df dataset with balls dataset

In [ ]:
balls.columns

In [ ]:
balls['BattingTeam'] = balls['BattingTeam'].str.replace('Kings XI Punjab', 'Punjab Kings')
balls['BattingTeam'] = balls['BattingTeam'].str.replace('Delhi Daredevils', 'Delhi Capitals')
balls['BattingTeam'] = balls['BattingTeam'].str.replace('Deccan Chargers', 'Sunrisers Hyderabad')

balls = balls[balls['BattingTeam'].isin(teams)]

In [ ]:
balls_df = match_df.merge(balls, on='ID')

In [ ]:
balls_df.head()

In [ ]:
balls_df['BattingTeam'].value_counts()

In [ ]:
fig = px.bar(balls_df['BattingTeam'].value_counts())
fig.show()

In [ ]:
balls_df.columns

Only select rows where we are in 2nd innings

In [ ]:
balls_df = balls_df[balls_df['innings']==2]

In [ ]:
balls_df.shape

In [ ]:
balls_df.head()

In [ ]:
balls_df.columns

## Create new row current_score after each ball

In [ ]:
balls_df['current_score'] = balls_df.groupby('ID')['total_run'].cumsum()

In [ ]:
balls_df

In [ ]:
balls_df['runs_left'] = np.where(balls_df['target']-balls_df['current_score']>=0, balls_df['target']-balls_df['current_score'], 0)

In [ ]:
balls_df

In [ ]:
balls_df['balls_left'] = np.where(120 - balls_df['overs']*6 - balls_df['ballnumber']>=0,120 - balls_df['overs']*6 - balls_df['ballnumber'], 0)

In [ ]:
balls_df['wickets_left'] = 10 - balls_df.groupby('ID')['isWicketDelivery'].cumsum()

In [ ]:
balls_df.columns

In [ ]:
balls_df['current_run_rate'] = (balls_df['current_score']*6)/(120-balls_df['balls_left'])

In [ ]:
balls_df['required_run_rate'] = np.where(balls_df['balls_left']>0, balls_df['runs_left']*6/balls_df['balls_left'], 0)

In [ ]:
balls_df.columns

In [ ]:
def result(row):
    return 1 if row['BattingTeam'] == row['WinningTeam'] else 0

In [ ]:
balls_df['result'] = balls_df.apply(result, axis=1)

In [ ]:
balls_df.head()

In [ ]:
balls_df.columns

In [ ]:
index1 = balls_df[balls_df['Team2']==balls_df['BattingTeam']]['Team1'].index
index2 = balls_df[balls_df['Team1']==balls_df['BattingTeam']]['Team2'].index

In [ ]:
balls_df.loc[index1, 'BowlingTeam'] = balls_df.loc[index1, 'Team1']
balls_df.loc[index2, 'BowlingTeam'] = balls_df.loc[index2, 'Team2']

In [ ]:
balls_df.head()

In [ ]:
final_df = balls_df[['BattingTeam', 'BowlingTeam','City','runs_left','balls_left','wickets_left','current_run_rate','required_run_rate','target','result']]

In [ ]:
final_df.head()

In [ ]:
fig = px.bar(final_df['City'].value_counts())
fig.show()

In [ ]:
fig = px.bar(final_df['BattingTeam'].value_counts())
fig.show()

In [ ]:
fig = px.bar(final_df['BowlingTeam'].value_counts())
fig.show()

In [ ]:
fig = px.histogram(final_df[final_df['runs_left']>0]['runs_left'], nbins=30,)
fig.show()

In [ ]:
fig = px.bar(final_df['wickets_left'].value_counts())
fig.show()

In [ ]:
fig = px.histogram(final_df['target'], nbins=30)
fig.show()

In [ ]:
final_df.describe()

In [ ]:
final_df.isna().sum()

In [ ]:
final_df.shape

In [ ]:
final_df.sample(final_df.shape[0])

Randomly shuffle all the rows

In [ ]:
final_df.sample()

## One hot encoding

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

trf = ColumnTransformer([
    ('trf', OneHotEncoder(sparse=False,drop='first'),['BattingTeam','BowlingTeam','City'])
],
remainder = 'passthrough')

In [ ]:
from sklearn.model_selection import train_test_split

X = final_df.drop('result', axis=1)
y = final_df['result']
X.shape, y.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, random_state=42)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

In [ ]:
pipe = Pipeline(steps=[
    ('step1',trf),
    ('step2',RandomForestClassifier())
])

In [ ]:
pipe.fit(X_train, y_train)

In [ ]:
y_pred = pipe.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_pred, y_test)

In [ ]:
pipe.predict_proba(X_test)

# Save model

In [ ]:
teams

In [ ]:
final_df['City'].unique()

In [ ]:
import pickle
pickle.dump(pipe, open('pipe.pkl','wb'))